# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,10.37,75,88,8.99,CL,1705721593
1,1,kolonia,50.9333,6.9500,-2.13,86,0,2.57,DE,1705721593
2,2,kerikeri,-35.2268,173.9474,26.62,72,43,2.24,NZ,1705721594
3,3,el granada,37.5027,-122.4694,14.39,91,100,4.12,US,1705721594
4,4,adamstown,-25.0660,-130.1015,24.05,84,27,6.95,PN,1705721595


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [86]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)
# Display the map
city_map

hvplot.show(city_map)



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [58]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_data_df = city_data_df[(city_data_df['Cloudiness'] == 0) & \
                                   (city_data_df['Max Temp'] >= 21) & \
                                   (city_data_df['Max Temp'] <= 27) & \
                                   (city_data_df['Wind Speed'] <= 4.5)]

# Drop any rows with null values
narrow_city_data_df = narrow_city_data_df.dropna(how='any')

# Display sample data
narrow_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,saint-pierre,-21.3393,55.4781,26.82,69,0,1.54,RE,1705721617
53,53,nicoya,10.1483,-85.4520,24.60,75,0,4.12,CR,1705721357
71,71,hermanus,-34.4187,19.2345,22.54,52,0,2.55,ZA,1705721637
103,103,ziguinchor,12.5833,-16.2719,21.11,88,0,3.09,SN,1705721658
152,152,pedernales,18.0384,-71.7440,25.05,62,0,1.25,DO,1705721692
222,222,cabo san lucas,22.8909,-109.9124,23.06,72,0,0.89,MX,1705721346
238,238,san ignacio,17.1561,-89.0714,24.08,100,0,2.06,BZ,1705721748
242,242,les avirons,-21.2418,55.3394,25.05,69,0,1.54,RE,1705721751
246,246,brisas de zicatela,15.8369,-97.0419,26.29,69,0,1.54,MX,1705721754
259,259,el porvenir,13.7617,-87.3458,22.63,65,0,2.73,HN,1705721762


### Step 3: Create a new DataFrame called `hotel_df`.

In [70]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_data_df[['City' , 'Country' , 'Lat' , 'Lng' ,'Humidity']]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

C:\Users\royip\AppData\Local\Temp\ipykernel_16744\576075356.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
40,saint-pierre,RE,-21.3393,55.4781,69,
53,nicoya,CR,10.1483,-85.4520,75,
71,hermanus,ZA,-34.4187,19.2345,52,
103,ziguinchor,SN,12.5833,-16.2719,88,
152,pedernales,DO,18.0384,-71.7440,62,
222,cabo san lucas,MX,22.8909,-109.9124,72,
238,san ignacio,BZ,17.1561,-89.0714,100,
242,les avirons,RE,-21.2418,55.3394,69,
246,brisas de zicatela,MX,15.8369,-97.0419,69,
259,el porvenir,HN,13.7617,-87.3458,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [76]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    "apiKey":geoapify_key,
    "categories":categories	
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
nicoya - nearest hotel: Hotel marianela
hermanus - nearest hotel: Aloe guest house
ziguinchor - nearest hotel: Hotel Ndary
pedernales - nearest hotel: Hostal Doña Chava
cabo san lucas - nearest hotel: Comfort Rooms
san ignacio - nearest hotel: Rainforest Haven Inn
les avirons - nearest hotel: Floralys
brisas de zicatela - nearest hotel: Casa de Olas
el porvenir - nearest hotel: No hotel found
kwolla - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
luanda - nearest hotel: Hotel Forum


,City,Country,Lat,Lng,Humidity,Hotel Name
40,saint-pierre,RE,-21.3393,55.4781,69,Tropic Hotel
53,nicoya,CR,10.1483,-85.4520,75,Hotel marianela
71,hermanus,ZA,-34.4187,19.2345,52,Aloe guest house
103,ziguinchor,SN,12.5833,-16.2719,88,Hotel Ndary
152,pedernales,DO,18.0384,-71.7440,62,Hostal Doña Chava
222,cabo san lucas,MX,22.8909,-109.9124,72,Comfort Rooms
238,san ignacio,BZ,17.1561,-89.0714,100,Rainforest Haven Inn
242,les avirons,RE,-21.2418,55.3394,69,Floralys
246,brisas de zicatela,MX,15.8369,-97.0419,69,Casa de Olas
259,el porvenir,HN,13.7617,-87.3458,65,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [85]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name" , "Country"]
)

# Display the map
city_hotel_map

hvplot.show(city_hotel_map)
